<h1>Pre-Trained Model InceptionResNetV2 with data augmentation</h1>

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [4]:
batch_size_training = 100
batch_size_validation = 100

In [5]:
image_resize = 224
num_classes = 2

In [7]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
train_generator = data_generator.flow_from_directory(
    'horse2zebra/training',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 3899 images belonging to 2 classes.


In [9]:
validation_generator = data_generator.flow_from_directory(
    'horse2zebra/validation',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 513 images belonging to 2 classes.


In [10]:
model = Sequential()

In [11]:
model.add(InceptionResNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [12]:
model.add(Dense(num_classes, activation='softmax'))
model.layers

In [13]:
model.layers[0].layers

In [14]:
model.layers[0].trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dense (Dense)                (None, 2)                 3074      
Total params: 54,339,810
Trainable params: 3,074
Non-trainable params: 54,336,736
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [17]:
#fit_history = model.fit_generator(
fit_history = model.fit(    
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
39/39 [==============================] - 387s 9s/step - loss: 0.6871 - accuracy: 0.5471 - val_loss: 0.6854 - val_accuracy: 0.5283
Epoch 2/2
39/39 [==============================] - 293s 8s/step - loss: 0.6754 - accuracy: 0.5719 - val_loss: 0.6703 - val_accuracy: 0.5867


In [18]:
model.save('classifier_inception_resnet_v2_model_2.h5')

In [19]:
test_generator = data_generator.flow_from_directory(
    'horse2zebra/testing',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 742 images belonging to 2 classes.


In [20]:
scores = model.evaluate_generator(test_generator,len(test_generator)) 
print("Loss = ", scores[0])
print("Accuracy = ", scores[1])
scores_inception_resnet_v2_augmentation = scores

/home/alexandr/.local/lib/python3.6/site-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Loss =  0.6782222390174866
Accuracy =  0.5633423328399658


In [22]:
model_inception_resnet_v2 = tensorflow.keras.models.load_model('classifier_inception_resnet_v2_model.h5')

In [23]:
scores = model_inception_resnet_v2.evaluate_generator(test_generator,len(test_generator)) 
print("Loss = ", scores[0])
print("Accuracy = ", scores[1])
scores_inception_resnet_v2 = scores

Loss =  0.7089605927467346
Accuracy =  0.5121293663978577


In [24]:
model_inception_resnet_v2_model_1 = tensorflow.keras.models.load_model('classifier_inception_resnet_v2_model_1.h5')

In [25]:
scores = model_inception_resnet_v2_model_1.evaluate_generator(test_generator,len(test_generator)) 
print("Loss = ", scores[0])
print("Accuracy = ", scores[1])
scores_inception_resnet_v2_enlarged = scores

Loss =  0.725234866142273
Accuracy =  0.48787063360214233


In [28]:
loss = list()

loss.append(scores_inception_resnet_v2[0])
loss.append(scores_inception_resnet_v2_enlarged[0])
loss.append(scores_inception_resnet_v2_augmentation[0])

accuracy = list()

accuracy.append(scores_inception_resnet_v2[1])
accuracy.append(scores_inception_resnet_v2_enlarged[1])
accuracy.append(scores_inception_resnet_v2_augmentation[0])

labels = ['inception_resnet_v2 (training examples = 2401)', 'inception_resnet_v2 (training examples = 3899)',
         'inception_resnet_v2 (with image augmentation)']

In [29]:
import pandas as pd
data = {'Model':labels, 'Loss':loss, 'Accuracy':accuracy}
test_df = pd.DataFrame(data)
test_df

,Model,Loss,Accuracy
0,inception_resnet_v2 (training examples = 2401),0.708961,0.512129
1,inception_resnet_v2 (training examples = 3899),0.725235,0.487871
2,inception_resnet_v2 (with image augmentation),0.678222,0.678222


| Model                                       	| Loss 	   | Accuracy |
|-----------------------------------------------|----------|----------|
|inception_resnet_v2 (training examples = 2401) | 0.016802 | 0.993261 |
|inception_resnet_v2 (training examples = 3899) | 0.014949 | 0.993261 |